# Preprocesamiento de Datos Estudiantiles: caso de estudio IEBS

## Metodología
- Paso 1: Limpieza de datos
- Paso 2: Transformar direcciones a coordenadas de latidud y longitud
- Paso 3: Normalización y escalado de datos
- Paso 4: Transformación de características estudiantiles basada en métricas

## Configuración Inicial

In [ ]:
%matplotlib notebook
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sb
from googlemaps import Client
import pylab
import pingouin as pg
import math
from math import sin, cos, sqrt, atan2, radians, asin, pi
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn.feature_selection import SelectKBest,chi2, mutual_info_classif, f_classif
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
import scipy.stats as stats
from scipy.stats import shapiro, ks_2samp, ks_1samp, kstest, chi2_contingency, norm

In [ ]:
dataset_ini = pd.read_csv('/Users/Downloads/Dataset/Final_Dataset/Data_original_IEBS.csv')
#dataset_ini.info()

### Paso 1: Limpieza de datos

- Se eliminan los registros vacíos e inconsistentes.
- Al final también se deben eliminar los datos atípico (ruido en los datos).

In [ ]:
missing_data = dataset_ini.isnull()

for column in missing_data.columns.values.tolist():
    print(column)
    print(missing_data[column].value_counts())
    print('')

In [ ]:
dataset_ini.replace('?', np.nan, inplace = True)
#dataset_ini.dropna(subset=['age'], axis = 0, inplace= True)
dataset_ini.reset_index(drop = True, inplace = True)

#dataset_ini['gender'].replace(np.nan, 'Top string', inplace = True)
#average_age = dataset_ini['age'].astype(float).mean(axis = 0)
#dataset_ini['course'].replace(np.nan, average_column_name, inplace = True)

### Paso 2: Transformar direcciones a coordenadas de latidud y longitud
- Se utiliza una clave para utilizar la API de google (tiene un costo) y georreferenciar las direcciones a partir de las características dirección de residencia y ciudad de residencia. Luego, se devuelve el valor en coordenadas de latitud y longitud. Es importante tener en cuenta que se debe especificar la ciudad junto con la dirección. Ejemplo: CL 104 #81 -13, medellín colombia. 

In [ ]:
# Convertir direcciones con la API de google
#KEY = 'AIzaSyCAHqWE9DbBOGsOjVxIojZMrYt_ZuxNoD' # Key para usar API de google (key de prueba)
gmaps = Client(key=KEY)

df_address = dataset_ini

def get_coordinates(address):
    #city = 'medellin, Colombia'
    #geocode_result = gmaps.geocode(str(address) +' '+ city)
    geocode_result = gmaps.geocode(str(address))
    if len(geocode_result) > 0:
        return list(geocode_result[0]['geometry']['location'].values())
    else:
        return [np.NaN, np.NaN]

coordinates = df_address['address'].apply(lambda x: pd.Series(get_coordinates(x), index=['LATITUDE', 'LONGITUDE']))
df_address = pd.concat([df_address[:], coordinates[:]], axis="columns")
df_address

### Paso 3: Normalización y escalado de datos

- Se utiliza el escalado estandar de los datos, este tipo de escalado de los datos, transforma cada valor en un rango alrededor de la media 0 y la desviación estándar 1, es decir, cada valor será escalado restando la media y dividiendo por la desviación estándar.
\begin{equation}
\label{equ:standard}
\begin{split}
\hspace{6cm} z& = \frac{x - u}{s}
\end{split}
\end{equation}

In [ ]:
scaler= StandardScaler()

dataset_ini_ = dataset_ini.drop(['Class'], axis=1) # quito la variable dependiente "Y"

scaler.fit(dataset_ini_) # calculo la media para poder hacer la transformacion
dataset_ini_scaled=scaler.transform(dataset_ini_)# Ahora si, escalo los datos

### Paso 4: Transformación de características estudiantiles basada en métricas (característica demográficas)
- Aquí se utilizan las características de latitud y longitud de ubicación de la residencia del estudiante para calcular la distancia que existe entre ese punto y la institución educativa.
Fórmula de Haversine:
    \begin{equation} d=2r\,sen^{-1}\Biggl(\sqrt{sen^2\biggl(\frac{lat2-lat1}{2}\biggr)+cos(lat1)\,cos(lat2)\,sen^2\biggl(\frac{lon2-lon1}{2}\biggr)}\Biggr)
    \end{equation}
- Se calcula la edad del estudiante con respecto a una fecha dada (fracción)
- Se calcula la extradad:
\begin{equation}
Edad_{teorica}=Grado_{ingreso} + 5
\end{equation}
\begin{equation}
Riesgo_{1}= Edad_{ingreso} - Edad_{teorica}
\end{equation}
- Se calcula la repitencia de grados
\begin{equation}
Riesgo_{2}= (2019 - Year_{in}  + Grado_{in}) -  Grado_{2019}
\end{equation}
- Se transforma los valores de la característica Puntaje_sisbén: Nivel 1: 0 $-$ 11, Nivel 2: 12 $-$ 22, Nivel 3: 23 $-$ 43, Nivel 4: 44 $-$ 65, Nivel 5: 66 $-$ 79, Nivel 6: 80 $-$ 100
\begin{equation}
Riesgo_{2}= (2019 - Year_{in}  + Grado_{in}) -  Grado_{2019}
\end{equation}
\begin{equation}
Riesgo_{3}= Nivel Sisben III
\end{equation}
- se binariza los valores de la característica de hermanos_colegio: si tiene hermanos = 1 sino = 0
\begin{equation}
Riesgo_{4}=binarizacion
\end{equation}


In [ ]:
import datetime as dt 
from datetime import datetime, date, time, timedelta
#import calendar
fech1_str = '8/11/2008'
#fech1 = dt.datetime(2010, 08, 01)
fech1 = datetime.strptime(fech1_str, '%d/%m/%Y').date()
fech2 = datetime.now().date()# criterio de medición (año)
#edad = round((fech2-fech1)/dt.timedelta(365,0,0,0),2)
#print(edad) 
edad_ = round((fech2-fech1)/dt.timedelta(365,5,49,12),2) #promedio de años comunes y bisiestos 
#print(edad_)

In [33]:
df_address = pd.read_csv('/Users/Downloads/Dataset/AcademicDATA/1/20201121/address/output_address_3017.csv') 
#df_address = pd.read_csv('/Users//Downloads/Dataset/Final_dataset/datasets/Data_1449_416.csv') 

In [47]:
# convert decimal degrees to radians 
df_address['lat1'] = df_address['latitude'].apply(math.radians)
df_address['lon1'] = df_address['longitude'].apply(math.radians)
lon2 = math.radians(-75.57433)
lat2 = math.radians(6.307670)
#lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
# haversine formula 
df_address['dlon'] = lon2 - df_address['lon1'] 
df_address['dlat'] = lat2 - df_address['lat1'] 

R = 6371 #radio de la tierra
d_1 = (np.sin((df_address['dlat'])/2))**2 + np.cos(df_address['lat1'])* np.cos(lat2) * (np.sin((df_address['dlon'])/2))**2
df_address['distance'] = 2 * R * np.arcsin(np.sqrt(d_1)) 

#df_address

In [ ]:
# visualizar datos atípicos con respecto a la ubicación geográfica para luego eliminarlos
lat = np.array(df_address['latitude'])
lon = np.array(df_address['longitude'])

media = lat.mean()
std_x = lat.std()*2
media_y =lon.mean()
std_y = lon.std()*2

print(media)
print(std_x)
print(media_y)
print(std_y)

colors = ['blue']*len(lat)

for index, x in enumerate(lat):
    if abs(x-media) > std_x:
        colors[index] = 'red'      
        
for index, x in enumerate(lon):
    if abs(x-media_y) > std_y:
        colors[index] = 'red'

plt.figure()   
plt.scatter(lat, lon, s=2, color=colors)
plt.axhline(media_y, color='k', linestyle='--')
plt.axvline(media, color='k', linestyle='--')
 
v=media     #x-position of the center
u=media_y    #y-position of the center
b=std_x     #radius on the x-axis
a=std_y    #radius on the y-axis
 
t = np.linspace(0, 2*pi, 100)
plt.plot(v+b*np.cos(t), u+a*np.sin(t))
 
plt.xlabel('Latitude')
plt.ylabel('Longitude')
plt.show()

### Paso 4: Transformación de características estudiantiles basada en métricas (característica académicas)
Como primera medida calcular el promedio entre características, así:
Promedio_CNM: se calculó el promedio entre el valor de las características de ciencias naturales y matemáticas, las cuales tenían una correlación de 0.60:
    \begin{equation} 
    \hspace{1cm} \frac{Ciencias \ Naturales + Mat}{2}
    \end{equation}
Promedio_ART: se calculó el promedio entre el valor de las características de artes y tecnología cuya correlación es de 0.47:
    \begin{equation} 
    \hspace{1cm} \frac{Artes + Tecno}{2}
    \end{equation}
Promedio_CSCP: se calculó el promedio entre el valor de las características de cátedra de paz y ciencias sociales, las cuales tenían una correlación de 0.61:
    \begin{equation} 
    \hspace{1cm} \frac{Catedra \ de \ Paz + Ciencias \ Sociales}{2}
    \end{equation}
Promedio_ESIN: se calculó el promedio entre el valor de las características de español inglés cuya correlación es de 0.55:
    \begin{equation} 
    \hspace{1cm} \frac{Esp + Ing}{2}
    \end{equation}
Promedio_ETR: se calculó el promedio entre el valor de las características de ética y religión, las cuales tenían una correlación de 0.55:
    \begin{equation} 
    \hspace{1cm} \frac{Etica + Rel}{2}
    \end{equation}
Promedio_Aaih: se calculó el promedio entre el valor de las asignaturas de alta intensidad horaria:
    \begin{equation} 
    \hspace{1cm}\frac{C.Naturales + Esp + Mat + C. Sociales + Ing}{5}
    \end{equation}
Promedio_Abih: se calculó el promedio entre el valor de las asignaturas de baja intensidad horaria:
    \begin{equation} 
    \hspace{1cm}\frac{Etica + Rel + Tecno + Empren+ Deport + Art + C. de Paz}{7}
    \end{equation}
Riesgo 1: riesgo de deserción con respecto a la asignatura de ciencias naturales:
    \begin{equation}
    \hspace{1cm}Riesgo_{1}= Valor_{CN} \times \frac{ Prom_{A_{aih}} + Prom_{A_{bih}}}{2}; \ Valor_{CN} = \frac{Nota_{CN}}{5}
    \end{equation}
Riesgo 2: riesgo de deserción con respecto a la asignatura de emprendimiento:
    \begin{equation}
    \hspace{1cm}Riesgo_{2}= Valor_{Emprend} \times \frac{Prom_{A_{aih}} + Prom_{A_{bih}}}{2} ; \ Valor_{Emp} = \frac{Nota_{Emp}}{5}
    \end{equation}
Riesgo 3:riesgo de deserción con respecto a la asignatura de matemáticas:
    \begin{equation}
    \hspace{1cm}Riesgo_{3}= Valor_{Mat} \times \frac{ Prom_{A_{aih}} + Prom_{A_{bih}}}{2}; \ Valor_{Mat} = \frac{Nota_{Mat}}{5}
    \end{equation}
Riesgo 4: riesgo de deserción con respecto a la asignatura de deportes:
    \begin{equation}
    \hspace{1cm}Riesgo_{4}= Valor_{Deport} \times \frac{Prom_{A_{aih}} + Prom_{A_{bih}}}{2}; \ Valor_{Dep} = \frac{Nota_{Dep}}{5}
    \end{equation}